# **calt Minimal Demo**

This notebook demonstrates the minimal code needed to:
1. Generate a dataset
2. Train a model
3. Evaluate the model

In [1]:
import sys
from pathlib import Path

# Add development calt to path (prioritize over pip-installed version)
# This notebook is in calt/examples/demos/, so we go up to calt/ and then to src/
# When running in Jupyter, Path.cwd() gives the notebook's directory
calt_dev_path = Path.cwd().parent.parent / "src"
sys.path.insert(0, str(calt_dev_path))

print(f"Using development calt from: {calt_dev_path}")

Using development calt from: /home/ara_shun/workspace/calt/src


## 1. Dataset Generation

Generate polynomial addition problems

In [2]:
import random

from calt.dataset.sympy.dataset_generator import DatasetGenerator
from calt.dataset.sympy.utils.polynomial_sampler import PolynomialSampler
from calt.dataset.utils.dataset_writer import DatasetWriter


# Define instance generator: polynomial addition
def polynomial_addition_generator(seed):
    random.seed(seed)

    # Initialize polynomial sampler
    sampler = PolynomialSampler(
        symbols="x0, x1",
        field_str="GF(7)",
        max_num_terms=2,
        max_degree=2,
        min_degree=1,
    )

    # Generate two polynomials
    F = sampler.sample(num_samples=2)

    # Solution is the sum
    g = sum(F)

    return F, g


# Initialize dataset generator
dataset_generator = DatasetGenerator(
    n_jobs=1,  # Use 1 for demo (SymPy backend)
    root_seed=100,
)

# Initialize dataset writer
dataset_writer = DatasetWriter(
    save_dir="./data",
    save_text=True,
    save_json=False,
)

# Generate datasets
dataset_generator.run(
    dataset_sizes={"train": 10000, "test": 100},
    instance_generator=polynomial_addition_generator,
    dataset_writer=dataset_writer,
)

print("Dataset generation completed!")

=========================== Dataset generation ===========================

Starting dataset generation for 2 dataset(s)
Dataset sizes: {'train': 10000, 'test': 100}

---------------------------------- train ----------------------------------
Dataset size: 10000 samples  (Batch size: 100000)

--- Batch 1/1 ---
Processing samples 1-10000 (size: 10000)
Starting parallel processing...
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1249 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 2449 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 3199 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 4999 tasks      | elapsed:    1.1s

Dataset generation completed!


## 2. Model Training

Load data, create model, and train

In [3]:
from omegaconf import OmegaConf

from calt.io import IOPipeline
from calt.models import ModelPipeline
from calt.trainer import TrainerPipeline

# Load config from YAML file
cfg = OmegaConf.load("./configs/config.yaml")

print("Config loaded")

Config loaded


In [4]:
# Load data
io_pipeline = IOPipeline.from_config(cfg.data)
# io_pipeline.dataset_load_preprocessor = dataset_load_preprocessor
result = io_pipeline.build()

print(
    f"Loaded {len(result['train_dataset'])} training samples and {len(result['test_dataset'])} test samples"
)

Loaded 10000 samples from ./data/train_raw.txt
Loaded 100 samples from ./data/test_raw.txt


Validating test dataset tokens... passed!
Loaded 10000 training samples and 100 test samples


In [5]:
# Create model
model_pipeline = ModelPipeline(cfg.model, result["tokenizer"])
model = model_pipeline.build()

print(
    f"Model: {type(model).__name__} ({sum(p.numel() for p in model.parameters()):,} parameters)"
)

Model: Transformer (1,002,496 parameters)


/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [6]:
# Create trainer
trainer_pipeline = TrainerPipeline(
    cfg.train,
    model=model,
    tokenizer=result["tokenizer"],
    train_dataset=result["train_dataset"],
    eval_dataset=result["test_dataset"],
    data_collator=result["data_collator"],
)
trainer = trainer_pipeline.build()

print("Trainer ready")

/home/ara_shun/workspace/calt/src/calt/trainer/trainer.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Trainer ready


In [7]:
# Train and evaluate
trainer.train()
eval_metrics = trainer.evaluate()
success_rate = trainer.evaluate_and_save_generation()

print(f"Success rate: {100 * success_rate:.1f}%")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/ara_shun/.netrc.
wandb: Currently logged in as: shun-arkw (chiba-u) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/wandb/analytics/sentry.py:268: DeprecationWarning: Read the `app_url` setting from the appropriate Settings object.
  app_url = wandb.util.app_url(tags["base_url"])  # type: ignore[index]
/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/wandb/analytics/sentry.py:268: DeprecationWarning: Read the `app_url` setting from the appropriate Settings object.
  app_url = wandb.util.app_url(tags["base_url"])  # type: ignore[index]
/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/wandb/analytics/sentry.py:279: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


{'loss': 3.3729, 'grad_norm': 2.5898165702819824, 'learning_rate': 3.121019108280255e-05, 'epoch': 0.1597444089456869}
{'loss': 2.717, 'grad_norm': 2.7342028617858887, 'learning_rate': 6.305732484076433e-05, 'epoch': 0.3194888178913738}
{'loss': 1.9186, 'grad_norm': 2.3831541538238525, 'learning_rate': 9.490445859872612e-05, 'epoch': 0.4792332268370607}
{'loss': 1.435, 'grad_norm': 3.7517452239990234, 'learning_rate': 9.701704545454547e-05, 'epoch': 0.6389776357827476}
{'loss': 1.1541, 'grad_norm': 4.151821613311768, 'learning_rate': 9.346590909090909e-05, 'epoch': 0.7987220447284346}
{'loss': 0.9795, 'grad_norm': 3.8127329349517822, 'learning_rate': 8.991477272727273e-05, 'epoch': 0.9584664536741214}
{'loss': 0.8628, 'grad_norm': 5.033078670501709, 'learning_rate': 8.636363636363637e-05, 'epoch': 1.1182108626198084}
{'loss': 0.779, 'grad_norm': 3.3890419006347656, 'learning_rate': 8.28125e-05, 'epoch': 1.2779552715654952}
{'loss': 0.7093, 'grad_norm': 5.244777679443359, 'learning_rate

Running evaluate_and_save_generation (step=1000, metric_key_prefix=eval)


{'eval_loss': 0.40444302558898926, 'eval_token_accuracy': 0.8731060606060606, 'eval_success_rate': 0.23, 'eval_runtime': 0.067, 'eval_samples_per_second': 1493.033, 'eval_steps_per_second': 59.721, 'epoch': 3.194888178913738}


...

{'eval_generation_success_rate': 0.23, 'eval_generation_step': 1000, 'epoch': 3.194888178913738}
{'loss': 0.4565, 'grad_norm': 6.167156219482422, 'learning_rate': 3.6647727272727274e-05, 'epoch': 3.3546325878594248}
{'loss': 0.4488, 'grad_norm': 5.304116725921631, 'learning_rate': 3.3096590909090915e-05, 'epoch': 3.5143769968051117}
{'loss': 0.4554, 'grad_norm': 4.9376983642578125, 'learning_rate': 2.954545454545455e-05, 'epoch': 3.6741214057507987}
{'loss': 0.454, 'grad_norm': 5.564621448516846, 'learning_rate': 2.5994318181818182e-05, 'epoch': 3.8338658146964857}
{'loss': 0.4406, 'grad_norm': 3.62054443359375, 'learning_rate': 2.244318181818182e-05, 'epoch': 3.9936102236421727}
{'loss': 0.4295, 'grad_norm': 4.212924957275391, 'learning_rate': 1.8892045454545457e-05, 'epoch': 4.15335463258786}
{'loss': 0.4435, 'grad_norm': 6.1024580001831055, 'learning_rate': 1.534090909090909e-05, 'epoch': 4.313099041533547}
{'loss': 0.4324, 'grad_norm': 4.131100177764893, 'learning_rate': 1.17897727

Running evaluate_and_save_generation (step=1565, metric_key_prefix=eval)


{'eval_loss': 0.35647696256637573, 'eval_token_accuracy': 0.8816287878787878, 'eval_success_rate': 0.25, 'eval_runtime': 0.0619, 'eval_samples_per_second': 1616.676, 'eval_steps_per_second': 64.667, 'epoch': 5.0}


Successfully saved generation results (step=1565, success_rate=0.2500)


{'eval_generation_success_rate': 0.25, 'eval_generation_step': 1565, 'epoch': 5.0}
Success rate: 25.0%


## 3. Summary

The entire training pipeline can be summarized in just a few lines:

In [8]:
# Complete minimal training code
from omegaconf import OmegaConf

from calt.io import IOPipeline
from calt.models import ModelPipeline
from calt.trainer import TrainerPipeline

# Load config from YAML file
cfg = OmegaConf.load("./configs/config.yaml")

# Load data
io_pipeline = IOPipeline.from_config(cfg.data)
result = io_pipeline.build()

# Create model
model = ModelPipeline(cfg.model, result["tokenizer"]).build()

# Create trainer and train
trainer = TrainerPipeline(
    cfg.train,
    model=model,
    tokenizer=result["tokenizer"],
    train_dataset=result["train_dataset"],
    eval_dataset=result["test_dataset"],
    data_collator=result["data_collator"],
).build()

trainer.train()
success_rate = trainer.evaluate_and_save_generation()
print(f"Success rate: {100 * success_rate:.1f}%")

Loaded 10000 samples from ./data/train_raw.txt
Loaded 100 samples from ./data/test_raw.txt


Validating test dataset tokens... passed!


/home/ara_shun/anaconda3/envs/calt-env/lib/python3.12/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
/home/ara_shun/workspace/calt/src/calt/trainer/trainer.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


{'loss': 3.4046, 'grad_norm': 2.5851714611053467, 'learning_rate': 3.121019108280255e-05, 'epoch': 0.1597444089456869}
{'loss': 2.7262, 'grad_norm': 2.558344841003418, 'learning_rate': 6.305732484076433e-05, 'epoch': 0.3194888178913738}
{'loss': 1.9646, 'grad_norm': 4.806286811828613, 'learning_rate': 9.490445859872612e-05, 'epoch': 0.4792332268370607}
{'loss': 1.4958, 'grad_norm': 3.0744497776031494, 'learning_rate': 9.701704545454547e-05, 'epoch': 0.6389776357827476}
{'loss': 1.196, 'grad_norm': 2.605043649673462, 'learning_rate': 9.346590909090909e-05, 'epoch': 0.7987220447284346}
{'loss': 0.9969, 'grad_norm': 3.7370448112487793, 'learning_rate': 8.991477272727273e-05, 'epoch': 0.9584664536741214}
{'loss': 0.8719, 'grad_norm': 4.18489408493042, 'learning_rate': 8.636363636363637e-05, 'epoch': 1.1182108626198084}
{'loss': 0.7842, 'grad_norm': 5.51294469833374, 'learning_rate': 8.28125e-05, 'epoch': 1.2779552715654952}
{'loss': 0.721, 'grad_norm': 6.80967903137207, 'learning_rate': 7.

Running evaluate_and_save_generation (step=1000, metric_key_prefix=eval)


{'eval_loss': 0.4075268507003784, 'eval_token_accuracy': 0.8664772727272727, 'eval_success_rate': 0.19, 'eval_runtime': 0.0632, 'eval_samples_per_second': 1582.434, 'eval_steps_per_second': 63.297, 'epoch': 3.194888178913738}


Successfully saved generation results (step=1000, success_rate=0.1900)


{'eval_generation_success_rate': 0.19, 'eval_generation_step': 1000, 'epoch': 3.194888178913738}
{'loss': 0.4641, 'grad_norm': 5.6160149574279785, 'learning_rate': 3.6647727272727274e-05, 'epoch': 3.3546325878594248}
{'loss': 0.4573, 'grad_norm': 3.478977918624878, 'learning_rate': 3.3096590909090915e-05, 'epoch': 3.5143769968051117}
{'loss': 0.4641, 'grad_norm': 4.71636962890625, 'learning_rate': 2.954545454545455e-05, 'epoch': 3.6741214057507987}
{'loss': 0.4609, 'grad_norm': 4.846352577209473, 'learning_rate': 2.5994318181818182e-05, 'epoch': 3.8338658146964857}
{'loss': 0.4457, 'grad_norm': 3.2434637546539307, 'learning_rate': 2.244318181818182e-05, 'epoch': 3.9936102236421727}
{'loss': 0.4421, 'grad_norm': 3.988615036010742, 'learning_rate': 1.8892045454545457e-05, 'epoch': 4.15335463258786}
{'loss': 0.4474, 'grad_norm': 4.46818208694458, 'learning_rate': 1.534090909090909e-05, 'epoch': 4.313099041533547}
{'loss': 0.4432, 'grad_norm': 4.820217132568359, 'learning_rate': 1.17897727